1. impot libraries
3. choose stocks and timeframe 
2. load data
3. transform to log returns
4. assign weights for portfolio return cretion
5. rolling predictions for parametric var and cvar with 252 days window and normal distribution
    1) 

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm, t

In [2]:
tickers = ['AAPL', 'JNJ', 'XOM', 'JPM', 'WMT']

data = yf.download(tickers, period='2y', interval='1d')['Close']

data = data.dropna()

closing_prices = data[-501:]

print(closing_prices)

/var/folders/g3/d8cr9sbj1nq0g6fwxyhphc380000gn/T/ipykernel_24929/3731822354.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period='2y', interval='1d')['Close']
[*********************100%***********************]  5 of 5 completed

Ticker            AAPL         JNJ         JPM        WMT         XOM
Date                                                                 
2023-08-03  189.274902  160.239288  149.164352  51.810757   99.989723
2023-08-04  180.185928  158.736816  148.849518  51.511463  100.269753
2023-08-07  177.077042  162.549362  149.555496  52.210907  100.064384
2023-08-08  178.017639  162.624466  148.715942  52.262955  100.559128
2023-08-09  176.423599  162.521164  146.721970  52.347538  102.267303
...                ...         ...         ...        ...         ...
2025-07-28  214.050003  166.220001  298.279999  97.610001  111.440002
2025-07-29  211.270004  168.110001  297.040009  98.330002  112.879997
2025-07-30  209.050003  167.259995  299.630005  97.589996  111.900002
2025-07-31  207.570007  164.740005  296.239990  97.980003  111.639999
2025-08-01  202.380005  167.330002  289.369995  98.489998  109.639999

[501 rows x 5 columns]


In [3]:
log_returns = np.log(closing_prices/closing_prices.shift(1)).dropna()

In [4]:
log_returns

Ticker,AAPL,JNJ,JPM,WMT,XOM
Date,,,,,
2023-08-04,-0.049211,-0.009421,-0.002113,-0.005793,0.002797
2023-08-07,-0.017404,0.023734,0.004732,0.013487,-0.002050
2023-08-08,0.005298,0.000462,-0.005629,0.000996,0.004932
2023-08-09,-0.008995,-0.000635,-0.013499,0.001617,0.016844
2023-08-10,-0.001235,-0.005214,-0.001496,-0.000499,0.005098
...,...,...,...,...,...
2025-07-28,0.000795,-0.012436,-0.001139,0.001435,0.009376
2025-07-29,-0.013073,0.011306,-0.004166,0.007349,0.012839
2025-07-30,-0.010563,-0.005069,0.008682,-0.007554,-0.008720


In [32]:
window = 250

alpha_95 = 0.95
alpha_99 = 0.99

z_score_95 = norm.ppf(alpha_95)
z_score_99 = norm.ppf(alpha_99)

weights = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

In [33]:
par_rolling_var_95 = []
par_rolling_var_99 = []

for t in range(window, len(log_returns)):
    window_log_returns = log_returns.iloc[t-window:t]
    cov_matrix = window_log_returns.cov().values
    portfolio_std = np.sqrt(weights.T @ cov_matrix @ weights)
    var_95 = z_score_95 * portfolio_std
    var_99 = z_score_99 * portfolio_std
    par_rolling_var_95.append(var_95) # assume mean = 0 due to negligible effect in daily log returns
    par_rolling_var_99.append(var_99)

In [34]:
par_var_95_series = pd.Series(par_rolling_var_95, index=log_returns.index[window:])
par_var_99_series = pd.Series(par_rolling_var_99, index=log_returns.index[window:])

In [35]:
realized_returns = log_returns @ weights
realized_returns = realized_returns[window:]

In [36]:
par_breaches_95 = realized_returns < -par_var_95_series
par_breaches_99 = realized_returns < -par_var_99_series

par_breach_rate_95 = par_breaches_95.sum()/len(par_breaches_95)
par_breach_rate_99 = par_breaches_99.sum()/len(par_breaches_99)

print(f'breach_rate_95: {par_breach_rate_95:.2%}')
print(f'breach_rate_99: {par_breach_rate_99:.2%}')

breach_rate_95: 6.40%
breach_rate_99: 3.20%


In [42]:
from scipy.stats import chi2

T = len(par_var_95_series)
x = len(par_breaches_95)
p = alpha_95  # nominal exception rate, e.g. 0.05

hat_p = x / T
L0 = ((1-p)**(T-x)) * (p**x)
L1 = ((1-hat_p)**(T-x)) * (hat_p**x)
LR_pof = -2 * np.log(L0 / L1)
p_value = 1 - chi2.cdf(LR_pof, df=1)

print(p_value)

4.1000723660644667e-07


In [ ]:
T = len(par_var_99_series)
x = len(par_breaches_99)
p = alpha_99  

hat_p = x / T
L0 = ((1-p)**(T-x)) * (p**x)
L1 = ((1-hat_p)**(T-x)) * (hat_p**x)
LR_pof = -2 * np.log(L0 / L1)
p_value = 1 - chi2.cdf(LR_pof, df=1)
print(p_value)

0.024981503053449705


In [63]:
hist_rolling_var_95 = []
hist_rolling_var_99 = []

port_log_returns = log_returns @ weights

for t in range(window, len(port_log_returns)):
    window_t = port_log_returns[t-window:t]
    var_t_95 = np.quantile(window_t, alpha_95)
    var_t_99 = np.quantile(window_t, alpha_99)
    hist_rolling_var_95.append(var_t_95)
    hist_rolling_var_99.append(var_t_99)

In [64]:
hist_var_95_series = pd.Series(hist_rolling_var_95, index=log_returns.index[window:])
hist_var_99_series = pd.Series(hist_rolling_var_99, index=log_returns.index[window:])

In [65]:
hist_breaches_95 = realized_returns < -hist_var_95_series
hist_breaches_99 = realized_returns < -hist_var_99_series

hist_breaches_rate_95 = hist_breaches_95.sum()/len(hist_breaches_95)
hist_breaches_rate_99 = hist_breaches_99.sum()/len(hist_breaches_99)

print(f'breach_rate_95: {hist_breaches_rate_95:.2%}')
print(f'breach_rate_99: {hist_breaches_rate_99:.2%}')

breach_rate_95: 6.00%
breach_rate_99: 2.80%


In [66]:
from scipy.stats import chi2
import numpy as np

T = len(hist_var_95_series)           # total number of observations
x = len(hist_breaches_95)         # number of VaR breaches (exceptions)
p = alpha_95                     # expected failure rate, e.g. 0.05

# Empirical exception rate
hat_p = x / T

# Avoid log(0) and division by zero
if hat_p == 0 or hat_p == 1:
    LR_pof = np.inf  # test is undefined in this case
    p_value = 0.0
else:
    # Kupiec likelihood ratio statistic
    L0 = (1 - p)**(T - x) * p**x
    L1 = (1 - hat_p)**(T - x) * hat_p**x
    LR_pof = -2 * np.log(L0 / L1)
    p_value = 1 - chi2.cdf(LR_pof, df=1)

print("LR PoF statistic:", LR_pof)
print("p-value:", p_value)


LR PoF statistic: inf
p-value: 0.0


In [62]:
from scipy.stats import chi2
import numpy as np

T = len(hist_var_99_series)           # total number of observations
x = len(hist_breaches_99)         # number of VaR breaches (exceptions)
p = alpha_99                     # expected failure rate, e.g. 0.01

# Empirical exception rate
hat_p = x / T

# Avoid log(0) and division by zero
if hat_p == 0 or hat_p == 1:
    LR_pof = np.inf  # test is undefined in this case
    p_value = 0.0
else:
    # Kupiec likelihood ratio statistic
    L0 = (1 - p)**(T - x) * p**x
    L1 = (1 - hat_p)**(T - x) * hat_p**x
    LR_pof = -2 * np.log(L0 / L1)
    p_value = 1 - chi2.cdf(LR_pof, df=1)

print("LR PoF statistic:", LR_pof)
print("p-value:", p_value)

LR PoF statistic: inf
p-value: 0.0


In [55]:
n_of_simulations = 10_000

mc_rolling_var_95 = []
mc_rolling_var_99 = []

for t in range(window, len(log_returns)):

    window_log_returns = log_returns.iloc[t-window:t]

    mu = window_log_returns.mean().values
    cov_matrix = window_log_returns.cov().values

    normal_simulation = np.random.multivariate_normal(mu, cov_matrix, n_of_simulations)
    sim_portfolio_returns = normal_simulation @ weights.T

    var_95 = -np.percentile(sim_portfolio_returns, 5)
    var_99 = -np.percentile(sim_portfolio_returns, 1)
    mc_rolling_var_95.append(var_95)
    mc_rolling_var_99.append(var_99)

In [56]:
mc_var_95_series = pd.Series(mc_rolling_var_95, index=log_returns.index[window:])
mc_var_99_series = pd.Series(mc_rolling_var_99, index=log_returns.index[window:])

In [57]:
mc_breaches_95 = realized_returns < -mc_var_95_series
mc_breaches_99 = realized_returns < -mc_var_99_series

mc_breach_rate_95 = mc_breaches_95.sum()/len(mc_breaches_95)
mc_breach_rate_99 = mc_breaches_99.sum()/len(mc_breaches_99)

print(f'breach_rate_95: {mc_breach_rate_95:.2%}')
print(f'breach_rate_99: {mc_breach_rate_99:.2%}')

breach_rate_95: 7.60%
breach_rate_99: 3.60%


In [58]:
from scipy.stats import chi2

T = len(mc_var_95_series)
x = len(mc_breaches_95)
p = alpha_95  # nominal exception rate, e.g. 0.05

hat_p = x / T
L0 = ((1-p)**(T-x)) * (p**x)
L1 = ((1-hat_p)**(T-x)) * (hat_p**x)
LR_pof = -2 * np.log(L0 / L1)
p_value = 1 - chi2.cdf(LR_pof, df=1)

print(p_value)

4.1000723660644667e-07


In [59]:
T = len(mc_var_99_series)
x = len(mc_breaches_99)
p = alpha_99

hat_p = x / T
L0 = ((1-p)**(T-x)) * (p**x)
L1 = ((1-hat_p)**(T-x)) * (hat_p**x)
LR_pof = -2 * np.log(L0 / L1)
p_value = 1 - chi2.cdf(LR_pof, df=1)

print(p_value)

0.024981503053449705
